# Day 16: Packet Decoder
https://adventofcode.com/2021/day/16 

The modules can be found [here](advent_of_code/day_16.py).

In [1]:
import math
from typing import Tuple

from advent_of_code.day_16 import Packet, LiteralPacket, OperatorPacket
from advent_of_code.utils import hex_to_binary, input_location

%load_ext autoreload
%autoreload 2

In [2]:
# Literals Test
p1 = Packet(hex_to_binary("D2FE28"))
assert isinstance(p1, LiteralPacket)
assert p1.header.version == 6
assert p1.value == "2021"
assert p1.length() == len(hex_to_binary("D2FE28")) - 3
p1

LiteralPacket(version=6, type_id=4, value=2021, bits_len=15, bits=101111111000101)

In [3]:
# Operator Packet: bit labeled I (0)
p2 = Packet(hex_to_binary('38006F45291200'))
assert isinstance(p2, OperatorPacket)
assert p2.header.version == 1
assert p2.header.type_id == 6
assert p2.operator_info.length_type_id == 0

assert len(p2.sub_packets) == 2
assert p2.sub_packets[0].value == '10'
assert p2.sub_packets[0].length() == 11

assert p2.sub_packets[1].value == '20'
assert p2.sub_packets[1].length() == 16

p2

OperatorPacket(version=1, type_id=6, operator_type=0, packet_size=49, bits=110100010100101001000100100)
	 -> LiteralPacket(version=6, type_id=4, value=10, bits_len=5, bits=01010)
	 -> LiteralPacket(version=2, type_id=4, value=20, bits_len=10, bits=1000100100)

In [4]:
# Operator Packet: bit labeled I (1)
p3 = Packet(hex_to_binary("EE00D40C823060"))
assert isinstance(p3, OperatorPacket)
assert p3.header.version == 7
assert p3.header.type_id == 3

assert len(p3.sub_packets) == 3
assert p3.sub_packets[0].value == '1'
assert p3.sub_packets[1].value == '2'
assert p3.sub_packets[2].value == '3'

p3

OperatorPacket(version=7, type_id=3, operator_type=1, packet_size=51, bits=010100000011001000001000110000011)
	 -> LiteralPacket(version=2, type_id=4, value=1, bits_len=5, bits=00001)
	 -> LiteralPacket(version=4, type_id=4, value=2, bits_len=5, bits=00010)
	 -> LiteralPacket(version=1, type_id=4, value=3, bits_len=5, bits=00011)

In [5]:
# 8A004A801A8002F478 represents an operator packet (version 4) which contains 
# an operator packet (version 1) which contains an operator packet (version 5) 
# which contains a literal value (version 6); this packet has a version sum of 
# 16

p4 = Packet(hex_to_binary("8A004A801A8002F478"))
assert isinstance(p4, OperatorPacket)
assert p4.header.version == 4
assert len(p4.sub_packets) == 1

assert p4.sub_packets[0].header.version == 1
assert isinstance(p4.sub_packets[0], OperatorPacket)
assert len(p4.sub_packets[0].sub_packets) == 1

assert p4.sub_packets[0].sub_packets[0].header.version == 5
assert isinstance(p4.sub_packets[0].sub_packets[0], OperatorPacket)
assert len(p4.sub_packets[0].sub_packets[0].sub_packets) == 1

assert isinstance(p4.sub_packets[0].sub_packets[0].sub_packets[0], LiteralPacket)
assert p4.sub_packets[0].sub_packets[0].sub_packets[0].header.version == 6

print(p4.sum_of_version_numbers())
assert p4.sum_of_version_numbers() == 16

p4

16


OperatorPacket(version=4, type_id=2, operator_type=1, packet_size=69, bits=001010100000000001101010000000000000101111010001111)
	 -> OperatorPacket(version=1, type_id=2, operator_type=1, packet_size=51, bits=101010000000000000101111010001111)
	 -> OperatorPacket(version=5, type_id=2, operator_type=0, packet_size=33, bits=11010001111)
	 -> LiteralPacket(version=6, type_id=4, value=15, bits_len=5, bits=01111)

In [6]:
# 620080001611562C8802118E34 represents an operator packet (version 3) which 
# contains two sub-packets; each sub-packet is an operator packet that 
# contains two literal values. This packet has a version sum of 12.

p5 = Packet(hex_to_binary("620080001611562C8802118E34"))

assert p5.header.version == 3
assert isinstance(p5, OperatorPacket)
# assert len(p5.sub_packets) == 2

print(p5.sum_of_version_numbers())
print(p5)

assert p5.sum_of_version_numbers() == 12

12
OperatorPacket(version=3, type_id=0, operator_type=1, packet_size=102, bits=000000000000000001011000010001010101100010110010001000000000100001000110001110001101)
	 -> OperatorPacket(version=0, type_id=0, operator_type=0, packet_size=44, bits=0001000101010110001011)
	 -> LiteralPacket(version=0, type_id=4, value=10, bits_len=5, bits=01010)
	 -> LiteralPacket(version=5, type_id=4, value=11, bits_len=5, bits=01011)
	 -> OperatorPacket(version=1, type_id=0, operator_type=1, packet_size=40, bits=0001000110001110001101)
	 -> LiteralPacket(version=0, type_id=4, value=12, bits_len=5, bits=01100)
	 -> LiteralPacket(version=3, type_id=4, value=13, bits_len=5, bits=01101)


In [7]:
with open(input_location(day=16)) as f:
    hex = f.read().strip()
    real_packet = Packet(hex_to_binary(hex))
    print(real_packet.sum_of_version_numbers())

917


### Part 2

Literal values (type ID 4) represent a single number as described above. The remaining type IDs are more interesting:

- Packets with type ID 0 are sum packets - their value is the sum of the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
- Packets with type ID 1 are product packets - their value is the result of multiplying together the values of their sub-packets. If they only have a single sub-packet, their value is the value of the sub-packet.
- Packets with type ID 2 are minimum packets - their value is the minimum of the values of their sub-packets.
- Packets with type ID 3 are maximum packets - their value is the maximum of the values of their sub-packets.
- Packets with type ID 5 are greater than packets - their value is 1 if the value of the first sub-packet is greater than the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
- Packets with type ID 6 are less than packets - their value is 1 if the value of the first sub-packet is less than the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.
- Packets with type ID 7 are equal to packets - their value is 1 if the value of the first sub-packet is equal to the value of the second sub-packet; otherwise, their value is 0. These packets always have exactly two sub-packets.

In [8]:
def decode_packet(p:Packet) -> int:
    if p.header.type_id == 4:
        return int(p.value)
    elif p.header.type_id == 0:
        return sum([decode_packet(sub_p) for sub_p in p.sub_packets])
    elif p.header.type_id == 1:
        return math.prod([decode_packet(sub_p) for sub_p in p.sub_packets])
    elif p.header.type_id == 2:
        return min([decode_packet(sub_p) for sub_p in p.sub_packets])
    elif p.header.type_id == 3:
        return max([decode_packet(sub_p) for sub_p in p.sub_packets])
    elif p.header.type_id == 5:
        sub_ps = [decode_packet(sub_p) for sub_p in p.sub_packets]
        assert len(sub_ps) == 2
        return 1 if sub_ps[0] > sub_ps[1] else 0
    elif p.header.type_id == 6:
        sub_ps = [decode_packet(sub_p) for sub_p in p.sub_packets]
        assert len(sub_ps) == 2
        return 1 if sub_ps[0] < sub_ps[1] else 0
    elif p.header.type_id == 7:
        sub_ps = [decode_packet(sub_p) for sub_p in p.sub_packets]
        assert len(sub_ps) == 2
        return 1 if sub_ps[0] == sub_ps[1] else 0

assert decode_packet(Packet(hex_to_binary("C200B40A82"))) == 3
assert decode_packet(Packet(hex_to_binary("04005AC33890"))) == 54
assert decode_packet(Packet(hex_to_binary("880086C3E88112"))) == 7
assert decode_packet(Packet(hex_to_binary("CE00C43D881120"))) == 9
assert decode_packet(Packet(hex_to_binary("D8005AC2A8F0"))) == 1
assert decode_packet(Packet(hex_to_binary("F600BC2D8F"))) == 0
assert decode_packet(Packet(hex_to_binary("9C005AC2F8F0"))) == 0
assert decode_packet(Packet(hex_to_binary("9C0141080250320F1802104A08"))) == 1


In [9]:
with open(input_location(day=16)) as f:
    hex = f.read().strip()
    print(decode_packet(Packet(hex_to_binary(hex))))


2536453523344
